In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tweepy
from tweepy import OAuthHandler
import re
import pandas as pd
import csv
!pip install Sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Authentikasi API Twitter

consumer_key = '<DROP CONSUMER KEY HERE>'
consumer_secret = '<DROP CONSUMER SECRET HERE>'
access_token = '<DROP ACCESS TOKEN HERE>'
access_secret = '<DROP ACCESS SECRET HERE>'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
# ambil 200 tweet terbaru dari akun pengguna twitter
# tweet yang diambil merupakan tweet berlabel bahasa indonesia dan bukan merupakan retweet murni (retweet tanpa komentar)

tweets = []
for tweet in tweepy.Cursor(api.user_timeline, id="<DROP ID TWITTER HERE>", tweet_mode="extended").items():
  if len(tweets) < 200:
    if hasattr(tweet, "retweeted_status"):
      continue
    else:
      if tweet.lang == 'in':
        tweets.append(tweet.full_text)

In [ ]:
len(tweets)

200

In [ ]:
# hapus hashtag, mention, link
def filter(string):
  string = re.sub(r'@[A-Za-z0-9_]+', '', string)
  string = re.sub(r'#[A-Za-z0-9_]+', '', string)
  string = re.sub(r'http[A-Za-z0-9_:/?.]+', '', string)
  string = string.replace('\n', '')
  string = re.sub(r'\\[A-Za-z0-9]+', '', string)
  return string

# mengubah emoji & emoticon ke dalam kelas emosi
def replace_emoji(string):
  kamus = pd.read_excel('/content/drive/My Drive/Data/emoji.csv')
  for j in range (len(kamus)):
    string = string.replace(kamus['emoji'][j], ' :'+kamus['kategori'][j]+':')
  return string

# melakukan proses case folding
def case_folding(string):
  return string.lower().strip()

# menghapus karakter selain huruf, angka, dan '-'
def cleaning(string):
  string = re.sub(r'[^a-zA-Z0-9-]', ' ', string)
  return string

# mengubah kata alay ke dalam bentuk formalnya
def normalization(string):
  tokens = string.split()
  kamus = pd.read_csv('/content/drive/My Drive/Data/colloquial-indonesian-lexicon.csv')
  text = []
  for token in tokens:
    for j in range(len(kamus)):
      if token == kamus['slang'][j]:
        token = token.replace(token, kamus['formal'][j])  
    text.append(token)
  string = ' '.join(text)
  return string

# menghapus angka
def cleaning_2(string):
  string = re.sub(r'[^a-zA-Z-]', ' ', string)
  return string

# menghapus stopword
def stopword_removal(string):
  factory = StopWordRemoverFactory()
  stopword = factory.create_stop_word_remover()
  string = stopword.remove(string)
  return string

# mengubah kata ke dalam bentuk dasarnya
def stemming(string):
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  string = stemmer.stem(string)
  return string

In [ ]:
# melakukan pre processing data
def prepro(list):
  tweets = []
  for string in list:
    string = filter(string)
    string = replace_emoji(string)
    string = case_folding(string)
    string = cleaning(string)
    string = normalization(string)
    string = cleaning_2(string)
    string = stopword_removal(string)
    string = stemming(string)
    token = nltk.tokenize.word_tokenize(string)
    tweets.append(token)
  return tweets

In [ ]:
hasil_prepro = prepro(tweets)

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing import sequence
import pickle

tk = Tokenizer()
file_ = open('/content/drive/My Drive/Data/tk.pk','rb')
tk = pickle.load(file_)

data_enc = tk.texts_to_sequences(hasil_prepro)
data_pred = sequence.pad_sequences(data_enc, maxlen=280, padding='post')

In [ ]:
print(data_pred)

[[15900   509     0 ...     0     0     0]
 [  509    25 34379 ...     0     0     0]
 [ 3576   124     2 ...     0     0     0]
 ...
 [ 5395   362     8 ...     0     0     0]
 [  509 10873     4 ...     0     0     0]
 [    0     0     0 ...     0     0     0]]


In [ ]:
from keras.models import load_model
import numpy as np

# melakukan proses prediksi
labelei = ['E', 'I']
labelns = ['N', 'S']
labelft = ['F', 'T']
labeljp = ['J', 'P']

modelei = load_model('/content/drive/My Drive/Live/ei_4_3.h5')
modelns = load_model('/content/drive/My Drive/Live/ns_2_3.h5')
modelft = load_model('/content/drive/My Drive/Live/ft_8_3.h5')
modeljp = load_model('/content/drive/My Drive/Live/jp_8_6.h5')

predictionsei = modelei.predict(data_pred)
predictionsns = modelns.predict(data_pred)
predictionsft = modelft.predict(data_pred)
predictionsjp = modeljp.predict(data_pred)

prediction_labels_ei=[]
prediction_labels_ns=[]
prediction_labels_ft=[]
prediction_labels_jp=[]

for i in range(len(predictionsei)):
  prediction_labels_ei.append(labelei[np.argmax(predictionsei[i])])
  prediction_labels_ns.append(labelns[np.argmax(predictionsns[i])])
  prediction_labels_ft.append(labelft[np.argmax(predictionsft[i])])
  prediction_labels_jp.append(labeljp[np.argmax(predictionsjp[i])])

In [ ]:
# hasil prediksi

E = prediction_labels_ei.count('E')
I = prediction_labels_ei.count('I')
print("E : ", E)
print("I : ", I)

N = prediction_labels_ns.count('N')
S = prediction_labels_ns.count('S')
print("N : ", N)
print("S : ", S)

F = prediction_labels_ft.count('F')
T = prediction_labels_ft.count('T')
print("F : ", F)
print("T : ", T)

J = prediction_labels_jp.count('J')
P = prediction_labels_jp.count('P')
print("J : ", J)
print("P : ", P)

E :  122
I :  78
N :  104
S :  96
F :  81
T :  119
J :  117
P :  83
